In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import quandl
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
%matplotlib inline

from matplotlib.colors import LinearSegmentedColormap
cm = LinearSegmentedColormap.from_list(
   name='test', 
   #colors=['red','white','green','white','red']
   colors=['#FF7575', '#00FF66'], N=100
)

def get_start_end_dates(days=10):
    end = datetime.today() 
    start = end - timedelta()
    return start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')



In [ ]:
eur_ecb_pairs = [
'EURUSD','EURGBP','EURAUD','EURNZD','EURJPY','EURCNY','EURCHF','EURCAD','EURMXN','EURINR','EURBRL',
'EURRUB','EURKRW','EURIDR','EURSEK','EURSGD','EURNOK','EURZAR','EURDKK','EURISK','EURCZK','EURHKD',
'EURTRY']

non_eur_pairs = [
'USD/JPY','GBP/USD','USD/CHF','USD/CAD','AUD/USD','NZD/USD','AUD/JPY','GBP/JPY','CHF/JPY','AUD/CAD','CAD/JPY',
'NZD/JPY','AUD/NZD','GBP/AUD','GBP/CHF','AUD/CHF','GBP/NZD','USD/INR','USD/CNY','USD/SGD','USD/HKD',
'USD/DKK','GBP/CAD','USD/SEK','USD/RUB','USD/TRY','USD/MXN','USD/ZAR','USD/KRW','CAD/CHF','NZD/CAD',
'NZD/CHF','USD/NOK', 'SEK/NOK','SEK/DKK','NOK/DKK','GBP/SEK']

rates_df = [quandl.get("ECB/" + pair, authtoken=API_KEY) for pair in eur_ecb_pairs]
rates_df = pd.concat(rates_df,axis=1)
rates_df.columns = eur_ecb_pairs

for pair in non_eur_pairs:
    print(pair)
    cross_pair = ['EUR' + i for i in pair.split('/')]
    rates_df[pair.replace('/','')] = rates_df[cross_pair[1]] / rates_df[cross_pair[0]]
  
rates_df.tail()

In [ ]:
rates_df.to_pickle('rates.pickle')

In [ ]:
df = rates_df[rates_df.index >= '2019-11-01']
rets = np.log(df/df.shift(1))
rets = rets.T
rets[rets.columns[0]] = 0
rets = rets.cumsum(axis=1) #CUMSUM()
rets.dropna(axis=0, inplace=True)

rets.columns = rets.columns.strftime('%d-%b')
rets = rets.sort_values(by=rets.columns[-1],ascending=False)
rets.style.format('{:,.2%}').background_gradient(cmap=cm,axis=1)

In [ ]:
def bolinger_bands(price, window_size1, window_size2, num_of_std):

    rolling_mean1 = price.rolling(window=window_size1).mean()
    rolling_mean2 = price.rolling(window=window_size2).mean()
    
    rolling_std1  = price.rolling(window=window_size1).std()
    rolling_std2  = price.rolling(window=window_size2).std()
   
    upper_band1 = rolling_mean1 + (rolling_std1*num_of_std)
    lower_band1 = rolling_mean1 - (rolling_std1*num_of_std)

    upper_band2 = rolling_mean2 + (rolling_std2*num_of_std)
    lower_band2 = rolling_mean2 - (rolling_std2*num_of_std)

    df = pd.concat([price, rolling_mean1, rolling_mean2, upper_band1, lower_band2, upper_band2, lower_band2], axis=1)
    df.columns = [price.columns[0], 
                  f'{window_size1}d_avg',
                  f'{window_size2}d_avg', 
                  f'+{num_of_std} std {window_size1}', 
                  f'-{num_of_std} std {window_size1}',
                  f'+{num_of_std} std {window_size2}', 
                  f'-{num_of_std} std {window_size2}']
    return df

temp = bolinger_bands(rates_df.filter(regex=("GBPSEK")),50, 200,2)
temp[temp.index.year >= 2017].plot(grid=True, figsize=(20,10))

In [ ]:
pd.DataFrame([(i,df[i].mean(),df[i].std(),df[i].skew(), df[i].kurt()) for i in df.columns], columns = ['Pair','Mean', 'Std', 'Skew', 'Kurt'])

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
def lin_reg_slope(ts): 
    x= np.arange(len(ts))
    y= ts
    gradient, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    mn=np.min(x)
    mx=np.max(x)
    x1=np.linspace(mn,mx,500)
    y1=gradient*x1+intercept
    plt.plot(x,y,'ob')
    plt.plot(x1,y1,'-r')
    plt.grid(True)
    plt.show()

In [ ]:
lin_reg_slope(df['EURUSD'])